In [ ]:
proj1 = "tratamento A"
proj2 = "tratamento B"
proj3 = "tratamento C"

# 01 - Imports

## 01.A - Installing Required Packages

## 01.B - Loading Packages

In [ ]:
import warnings

warnings.filterwarnings("ignore")

from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.decomposition import PCA

from scipy.stats import ks_2samp,wasserstein_distance

import pickle
import numpy as np
import pandas as pd
import time
from functools import partial

import matplotlib.pyplot as plt
import seaborn as sns
from plotly import tools
import plotly.graph_objects as go

import tensorflow as tf
tf.compat.v1.disable_eager_execution()
from tensorflow.keras import models, layers, optimizers
from tensorflow.keras import applications
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Input, Dense, Lambda
from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard, EarlyStopping
from tensorflow.keras import regularizers
from tensorflow.keras.losses import mse, binary_crossentropy
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from tensorflow.keras import backend as K
from tensorflow.keras.layers import Dense, Flatten, Conv1D

from hyperopt import fmin, tpe, hp, STATUS_OK, Trials, space_eval

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import mean_squared_error
import pandas as pd
import numpy as np

## 01.C - Custom Classes and Functions

In [ ]:
# Custom Classes and Functions
from classes import xplor, LowerCase,MissingValue, Outliers, CharEncoder, ScalingTreatmeant, Autoencoder, compare_metric

In [ ]:
def build_model(params):
   
    l = [params[s] for s in params.keys() if s.startswith('unit')]

    return Autoencoder(
        layers_= l +  [params['center']],
        activations=[params['activation']]*len(l) + [params['center_activation']],
        final_act=params['final_activation'],
        loss="mean_squared_error",
        optimizer="adam",
        learning_rate = params['learning_rate']
    )

In [ ]:
def uni_plots(df_, df_synth, title):

    # Inputs para gerar o grid de gráficos
    n_cols = X_test_t2.shape[1]

    plot_r = int(np.ceil((n_cols/4)))
    plot_c = 4

    c = 0
    r = 0

    # for loop para plotar todas as variáveis
    W_List = {}
    fig, ax = plt.subplots(plot_r , plot_c, figsize = (4*plot_c,3*plot_r))
    for column in df_.columns:
        W_List[column] = []

        if (df_[column].dtype == 'object') | pd.CategoricalDtype.is_dtype(df_[column]):

            gb = df_.groupby([column]).size()
            gb = np.round((gb/gb.sum())*100,1)

            gb_s = df_synth.groupby([column]).size()
            gb_s = np.round((gb_s/gb_s.sum())*100,1)

            compare = pd.concat([gb,gb_s], axis = 1).fillna(0)
            compare.columns = ['Real','Synthetic']
            compare['DIFF'] = np.round(abs(compare['Real'] - compare['Synthetic']),2)

            compare[['Real','Synthetic']].plot(kind = 'bar', ax = ax[r,c], alpha = 0.8)
            ax[r,c].set_title(column+":"+ str(compare['DIFF'].abs().sum()))
            ax[r,c].legend()

            c+=1

            if c > 3:
                c = 0
                r+=1

            le = LabelEncoder()
            le.fit(df_[column])

            mms = MinMaxScaler()
            mms.fit(le.transform(df_[column]).reshape(-1,1))

            W_List[column].append(wasserstein_distance( np.squeeze(mms.transform(le.transform(df_synth[column].dropna()).reshape(-1,1)))
                                               ,np.squeeze(mms.transform(le.transform(df_      [column]         ).reshape(-1,1)))
                                              )
                         )

        else:           
            bins = np.histogram_bin_edges(df_[column], 20)
            sns.histplot(data=df_  , x=column, bins = bins, ax = ax[r,c], color = 'blue', alpha = 0.5, label = 'Real')
            sns.histplot(data=df_synth, x=column, bins = bins, ax = ax[r,c], color = 'red', alpha = 0.4, label = 'Synthetic')
            ax[r,c].set_title(column)
            ax[r,c].legend()

            c+=1
            if c > 3:
                c = 0
                r+=1



            mms = MinMaxScaler()
            mms.fit(df_[[column]])

            W_List[column].append(wasserstein_distance( np.squeeze(mms.transform(df_synth[[column]]))
                                               ,np.squeeze(mms.transform(df_[[column]]))
                                              )
                         )
    plt.suptitle(title, fontsize = 15)
    plt.tight_layout()
    

In [ ]:
def uni_plots2(df_, df_synth, title, c_size = 4, r_size = 4):

    # Inputs para gerar o grid de gráficos
    n_cols = X_test_t2.shape[1]

    plot_r = int(np.ceil((n_cols/4)))
    plot_c = 4

    c = 0
    r = 0

    # for loop para plotar todas as variáveis
    W_List = {}
    fig, ax = plt.subplots(plot_r , plot_c, figsize = (c_size*plot_c,r_size*plot_r))
    for column in df_.columns:
        W_List[column] = []

        if (df_[column].dtype == 'object') | pd.CategoricalDtype.is_dtype(df_[column]):

            gb = df_.groupby([column]).size()
            gb = np.round((gb/gb.sum())*100,1)

            gb_s = df_synth.groupby([column]).size()
            gb_s = np.round((gb_s/gb_s.sum())*100,1)

            compare = pd.concat([gb,gb_s], axis = 1).fillna(0)
            compare.columns = ['Real','Synthetic']
            compare['DIFF'] = np.round(abs(compare['Real'] - compare['Synthetic']),2)

            compare[['Real','Synthetic']].plot(kind = 'bar', color=['blue','red'], ax = ax[r,c], alpha = 0.8, legend = False)
            ax[r,c].set_title(column)
            ax[r,c].set_xticks([])
            ax[r,c].set_yticks([])
            ax[r,c].set_xlabel("")
            ax[r,c].set_ylabel("")
            c+=1

            if c > 3:
                c = 0
                r+=1

            le = LabelEncoder()
            le.fit(df_[column])

            mms = MinMaxScaler()
            mms.fit(le.transform(df_[column]).reshape(-1,1))

            W_List[column].append(wasserstein_distance( np.squeeze(mms.transform(le.transform(df_synth[column].dropna()).reshape(-1,1)))
                                               ,np.squeeze(mms.transform(le.transform(df_      [column]         ).reshape(-1,1)))
                                              )
                         )

        else:           
            bins = np.histogram_bin_edges(df_[column], 20)
            sns.histplot(data=df_  , x=column, bins = bins, ax = ax[r,c], color = 'blue', alpha = 0.5)
            sns.histplot(data=df_synth, x=column, bins = bins, ax = ax[r,c], color = 'red', alpha = 0.4)
            ax[r,c].set_title(column)
            ax[r,c].set_xlabel("")            
            ax[r,c].set_xticks([])
            ax[r,c].set_yticks([])
            ax[r,c].set_ylabel("")
            c+=1
            if c > 3:
                c = 0
                r+=1



            mms = MinMaxScaler()
            mms.fit(df_[[column]])

            W_List[column].append(wasserstein_distance( np.squeeze(mms.transform(df_synth[[column]]))
                                               ,np.squeeze(mms.transform(df_[[column]]))
                                              )
                         )
    plt.suptitle(title, fontsize = 15)

    
    

In [ ]:


def compare_models(real, synth):
    
    ce = CharEncoder()
    
    RX_m = real.drop([ 'salary'], axis = 1).copy()
    RX_m = ce.fit_transform(RX_m.copy())
    RY_m = real[['salary']].copy()

    X_trainR, X_testR, Y_trainR, Y_testR  = train_test_split(RX_m, RY_m , test_size=0.2, random_state=42)

    RFC_R = RandomForestClassifier()
    RFC_R.fit(X_trainR,Y_trainR)

    SX_m = synth.drop([ 'salary'], axis = 1).copy()
    SX_m = ce.transform(SX_m.copy())
    SX_m.index = RX_m.index

    SY_m = synth[['salary']].copy()
    SY_m.index = RY_m.index

    X_trainS = SX_m.loc[X_trainR.index,:]
    X_testS  = SX_m.loc[X_testR.index,:]
    Y_trainS = SY_m.loc[Y_trainR.index,:]
    Y_testS  = SY_m.loc[Y_testR.index,:]

    RFC_S = RandomForestClassifier(max_depth = 8)
    RFC_S.fit(X_trainS,Y_trainS)

    import numpy as np
    from sklearn.metrics import roc_auc_score
    
    
    final_predict = pd.DataFrame({'REAL':RFC_S.predict(X_testR), "SYNT":RFC_S.predict(X_testS)}, index = X_testR.index)
    final_predict['CHECK'] = final_predict["REAL"] == final_predict["SYNT"]
    
    real_predict = pd.DataFrame({'PRED':RFC_S.predict(X_testR), "REAL":Y_testR['salary']}, index = X_testR.index)
    real_predict['CHECK'] = real_predict["REAL"] == real_predict["PRED"]
    
    real_predict2 = pd.DataFrame({'PRED':RFC_S.predict_proba(X_testR)[:, 1], "REAL":Y_testR['salary']}, index = X_testR.index)
    
    return [final_predict['CHECK'].sum()/final_predict.shape[0] , real_predict['CHECK'].sum()/real_predict.shape[0], roc_auc_score(real_predict2.REAL, real_predict2.PRED)]

In [ ]:


def compare_models_train(real, synth):
    
    ce = CharEncoder()
    
    RX_m = real.drop([ 'salary'], axis = 1).copy()
    RX_m = ce.fit_transform(RX_m.copy())
    RY_m = real[['salary']].copy()

    X_trainR, X_testR, Y_trainR, Y_testR  = train_test_split(RX_m, RY_m , test_size=0.2, random_state=42)

    RFC_R = RandomForestClassifier()
    RFC_R.fit(X_trainR,Y_trainR)

    SX_m = synth.drop([ 'salary'], axis = 1).copy()
    SX_m = ce.transform(SX_m.copy())
    SX_m.index = RX_m.index

    SY_m = synth[['salary']].copy()
    SY_m.index = RY_m.index

    X_trainS = SX_m.loc[X_trainR.index,:]
    X_testS  = SX_m.loc[X_testR.index,:]
    Y_trainS = SY_m.loc[Y_trainR.index,:]
    Y_testS  = SY_m.loc[Y_testR.index,:]

    RFC_S = RandomForestClassifier(max_depth = 8)
    RFC_S.fit(X_trainS,Y_trainS)

    import numpy as np
    from sklearn.metrics import roc_auc_score
    
    
    final_predict = pd.DataFrame({'REAL':RFC_S.predict(X_trainR), "SYNT":RFC_S.predict(X_trainS)}, index = X_trainR.index)
    final_predict['CHECK'] = final_predict["REAL"] == final_predict["SYNT"]
    
    real_predict = pd.DataFrame({'PRED':RFC_S.predict(X_trainR), "REAL":Y_trainR['salary']}, index = X_trainR.index)
    real_predict['CHECK'] = real_predict["REAL"] == real_predict["PRED"]
    
    real_predict2 = pd.DataFrame({'PRED':RFC_S.predict_proba(X_trainR)[:, 1], "REAL":Y_trainR['salary']}, index = X_trainR.index)
    
    return [final_predict['CHECK'].sum()/final_predict.shape[0] , real_predict['CHECK'].sum()/real_predict.shape[0], roc_auc_score(real_predict2.REAL, real_predict2.PRED)]

# 02 - Data Reading

In [ ]:
#od.download("https://www.kaggle.com/rameshmehta/credit-risk-analysis/version/1")
data = pd.read_csv("adult.csv", header = None)
data.columns = ['age','workclass','fnlwgt','education','education_num','marital_status','occupation','relationship','race','sex','capital_gain','capital_loss','hours_per_week','native_country', 'salary']

In [ ]:
data.shape

In [ ]:
data.head()

In [ ]:
data[['fnlwgt']].sum()

# 03 - Data Cleansing

In [ ]:
# Criando classe para fazer limepza dos dados de forma automatizada
# Essa classe consegue analisar e identificar variáveis com baixa qualidade em relação a valores nulos,
# variáveis categóricas com muitas variáveis, variáveis do tipo data, variáveis com altíssima variância 
# e variáveis com variância nula
xp = xplor(data)

In [ ]:
# Checando variáveis nulas.
# É realizado um gráfico de pareto para a % de nulos em cada variável.

# De acordo com o parâmetro 'level', serão selecionadas (para exclusão) as variaveis que 
# ultrapassarem o valor desse parâmetro. Ou seja, nesse acso todas as variáveis com mais de 50%
# de nulos serão selecionadas para exclusão

# Foram encontradas 21 variáveis.

xp.check_nulls(level = 0, select = True)
print(xp.nulls)

In [ ]:
# Este próximo método visa identificar variáveis categóricas que possuem uma quantidade alta de valores únicos,
# Neste exemplo, toda variável categórica com mais de 20 categrias distintas será selecionada para exclusão.

# Neste caso foram encontradas 10 variáveis.

# O Gráfico de pareto é mostrado para ajudar na identificação visual

xp.check_unique_objects(level_unique = 50,select = True)

In [ ]:
# O próximo método visa identificar variáveis do tipo data/
# Para este experimento, essas variáveis serão excluídas.

# Neste exemplo, foram encontradas 5 variáveis

xp.check_dates(select = True)

In [ ]:
# Neste último método, o objetivo é encontrar variáveis que possuam uma variância normalizada muito alta ou nula.
# O interessante foi verificar que ela se mostrou últi para encontrar 
# as duas colunas relacionadas ao ID (com altíssima variância) e uma coluna sem variância.

xp.check_var(select = True)

In [ ]:
# Para finalizar esse processo, é executado o método 'clean_data',
# que vai pegar todas as variáveis identificadas nos métodos anteriores e vai excluí-las da base final

new_df = xp.clean_data()

In [ ]:
# Com isso, foram excluídas 34 variáveis.
# A base final agora possúi 39 variáveis.
new_df.shape

In [ ]:
new_df.groupby(['education_num','education']).size().reset_index()

In [ ]:
from matplotlib.pylab import *
from mpl_toolkits.axes_grid1 import host_subplot

df = data.copy()
plot_number = 10
numerics = df.select_dtypes(exclude = ['object','category']).columns
objects  = df.select_dtypes(include = ['object','category']).columns
    
mms = MinMaxScaler()

num_desc = pd.DataFrame(mms.fit_transform(df[numerics]), columns = numerics).describe()

obj_desc = df[objects].describe()

f0 = figure(num = 0, figsize = (12, 8))#, dpi = 300)
#f0.suptitle("Oscillation decay", fontsize=14)
ax01 = subplot2grid((2, 2), (0, 0))
ax02 = subplot2grid((2, 2), (0, 1))
ax03 = subplot2grid((2, 2), (1, 0), colspan=2, rowspan=1)

num_desc.loc['std',:].sort_values(ascending = False)[:plot_number].plot(kind = 'bar', ax = ax01)
ax01.set_ylabel("Variância Normalizada", fontdict = {'size':15})
ax01.set_xlabel("Variáveis", fontdict = {'size':15})
ax01.set_title('Variáveis com maior variância normalizada', fontdict = {'size':15})
ax01.tick_params(axis='x', labelsize=12, labelrotation = 45)

num_desc.loc['std',:].sort_values(ascending = True)[:plot_number].plot(kind = 'bar', ax = ax02)
ax02.set_ylabel("Variância Normalizada", fontdict = {'size':15})
ax02.set_xlabel("Variáveis", fontdict = {'size':15})
ax02.set_title('Variáveis com menor variância normalizada', fontdict = {'size':15})
ax02.tick_params(axis='x', labelsize=12, labelrotation = 45)

obj_desc.loc['unique',:].sort_values(ascending = False)[:20].plot(kind = 'bar', ax = ax03, label = 'Quantidades')
ax03.set_ylabel("# Categorias", fontdict = {'size':15})
ax03.set_xlabel("Variáveis", fontdict = {'size':15})
ax03.set_title('Variáveis com mais categorias', fontdict = {'size':15})
ax03.tick_params(axis='x', labelsize=12, labelrotation = 45)
f0.tight_layout()

# 04 - Data Treatmeant Pipeline (Pre-Processing)

In [ ]:
new_df.columns

In [ ]:
new_df.drop(['education_num'], axis = 1, inplace = True)

In [ ]:
import joblib


pipe_full = joblib.load('pipeline_'+proj1+'.sav')
pipe_out15 = joblib.load('pipeline_'+proj2+'.sav')
pipe_out30 = joblib.load('pipeline_'+proj3+'.sav')

# 05 - Train Test Split + Transformation

In [ ]:
# Criação da amostra treino e teste.
# Nesse caso existe apenas as bases 'X', não sendo necessário a base 'y', pois o input é o próprio target.
# O objetivo do processo é replicar exatamente os dados de entrada.

X_train, X_test = train_test_split(new_df.copy(), test_size=0.2, random_state=42)

# Aqui estamos aplicando o .transform() doa pipeline treinada.
# Aqui os dados sofrerão as transformações necessárias para ficarem prontos para o treinamento
# da rede neural.

# Estou mudando um pequeno parâmetro da pipeline para ter a exclusão dos outliers para a amostra treino
#my_pipe.set_params(Outliers__train=True)
pipe_out15.set_params(Outliers__train=True)
pipe_out30.set_params(Outliers__train=True)
X_train_full = pipe_full.transform(X_train.copy())
X_train_out15 = pipe_out15.transform(X_train.copy())
X_train_out30 = pipe_out30.transform(X_train.copy())

# Estou mudando um pequeno parâmetro da pipeline para não ter a exclusão dos outliers na amostra teste
#my_pipe.set_params(Outliers__train=False)
pipe_out15.set_params(Outliers__train=False)
pipe_out30.set_params(Outliers__train=False)
X_test_full = pipe_full.transform(X_test.copy())
X_test_out15 = pipe_out15.transform(X_test.copy())
X_test_out30 = pipe_out30.transform(X_test.copy())

In [ ]:
X_train.shape

In [ ]:
print("X_train_full ",X_train_full.shape)
print("X_train_out15",X_train_out15.shape)
print("X_train_out30",X_train_out30.shape)


print("X_test_full ",X_test_full.shape)
print("X_test_out15",X_test_out15.shape)
print("X_test_out30",X_test_out30.shape)


# 06 - Models

In [ ]:
all_dtypes = {i:str(X_train.dtypes[i]) for i in X_train.dtypes.index}

In [ ]:
#model 1 - AutoEncoder (AE)
# Carregando e treinando o melhor AE
print("Training Model FULL - AutoEncoder (AE)")
with open( proj1 + "_sparse_1hl_params_model.pkl", "rb") as f:
    params_full =  pickle.load(f)
ae_full = build_model(params_full)
ae_full.fit(X_train_full, epochs = 700)
synthetic_full = pipe_full.inverse_transform(ae_full.transform(X_test_full.copy())[0])
synthetic_full = synthetic_full.astype(all_dtypes)

In [ ]:
params_full

In [ ]:
#model 1 - AutoEncoder (AE)
# Carregando e treinando o melhor AE
print("Training Model FULL out 1.5 - AutoEncoder (AE)")
with open( proj2 + "_sparse_1hl_params_model.pkl", "rb") as f:
    params_out15 =  pickle.load(f)
ae_out15 = build_model(params_out15)
ae_out15.fit(X_train_out15, epochs = 700)
synthetic_out15 = pipe_out15.inverse_transform(ae_out15.transform(X_test_out15.copy())[0])
synthetic_out15 = synthetic_out15.astype(all_dtypes)

In [ ]:
params_out15

In [ ]:
#model 1 - AutoEncoder (AE)
# Carregando e treinando o melhor AE
print("Training Model FULL out 3.0 - AutoEncoder (AE)")
with open( proj3 + "_sparse_1hl_params_model.pkl", "rb") as f:
    params_out30 =  pickle.load(f)
ae_out30 = build_model(params_out30)
ae_out30.fit(X_train_out30, epochs = 700)
synthetic_out30 = pipe_out30.inverse_transform(ae_out30.transform(X_test_out30.copy())[0])
synthetic_out30 = synthetic_out30.astype(all_dtypes)

In [ ]:
params_out30

## 07.B - Fidelity

In [ ]:
X_test_t2 = pipe_full.inverse_transform(X_test_full.copy())

In [ ]:
X_test_t2.shape

In [ ]:
# Calculando a distância de Wasserstein para todas as colunas para os três modelos
W_df_tst = compare_metric(df_real = X_test_t2
                      ,dfs_synth = [synthetic_full, synthetic_out15, synthetic_out30]
                      , metric = 'wasserstein')

In [ ]:
# Data_0: AutoEncoder
# Data_1: Sparse AutoEncoder
# Data_2: Convolutional AutoEncoder
W_df_tst

In [ ]:
W_df_tst.quantile(q = [0,0.5,1]).append( pd.DataFrame(W_df_tst.mean(), columns = ['avg']).T)

In [ ]:
synthetic_full.columns

In [ ]:
def uni_plots3(df_, df_synth, title):

    # Inputs para gerar o grid de gráficos
    n_cols = X_test_t2.shape[1]

    plot_c = 2
    plot_r = int(np.ceil((n_cols/plot_c)))
    

    c = 0
    r = 0

    # for loop para plotar todas as variáveis
    W_List = {}
    for column in df_.columns:
        W_List[column] = []

        if (df_[column].dtype == 'object') | pd.CategoricalDtype.is_dtype(df_[column]):

            gb = df_.groupby([column]).size()
            gb = np.round((gb/gb.sum())*100,1)

            gb_s = df_synth.groupby([column]).size()
            gb_s = np.round((gb_s/gb_s.sum())*100,1)

            
            
            
            compare = pd.concat([gb,gb_s], axis = 1).fillna(0)
            compare.columns = ['Real','Synthetic']
            compare['DIFF'] = np.round(abs(compare['Real'] - compare['Synthetic']),2)

            
            #sns.swarmplot(data=compare, x="Real", y="Synthetic", ax = ax[r,c])
            
            fig, ax = plt.subplots(1 , 1, figsize = (5,5))
            compare[['Real','Synthetic']].plot(kind = 'barh', ax = ax, alpha = 0.8)
            ax.set_title(column+":"+ str(compare['DIFF'].abs().sum()))
            ax.legend()
            fig.tight_layout()
            fig.savefig(column+'2.png', )

            le = LabelEncoder()
            le.fit(df_[column])

            mms = MinMaxScaler()
            mms.fit(le.transform(df_[column]).reshape(-1,1))

            W_List[column].append(wasserstein_distance( np.squeeze(mms.transform(le.transform(df_synth[column].dropna()).reshape(-1,1)))
                                               ,np.squeeze(mms.transform(le.transform(df_      [column]         ).reshape(-1,1)))
                                              )
                         )

        else:    
            plt.savefig("marginal_plot_with_focus_on_marginals_Seaborn.png",figsize=(4,4), dpi=150)

            sns.jointplot(x=df_[column], 
                          y=df_synth[column],
                          edgecolor="white",height = 4, marginal_ticks = False
                          ,marginal_kws=dict(bins=20)
                         ).set_axis_labels(column+' - real', column+' - synthetic', fontsize=16)
            #plt.savefig(column+'.png')
            
            #bins = np.histogram_bin_edges(df_[column], 20)
            #sns.histplot(data=df_  , x=column, bins = bins, ax = ax[r,c], color = 'blue', alpha = 0.5, label = 'Real')
            #sns.histplot(data=df_synth, x=column, bins = bins, ax = ax[r,c], color = 'red', alpha = 0.4, label = 'Synthetic')
            #ax.set_title(column)
            #ax[r,c].legend()

            c+=1
            if c > plot_c-1:
                c = 0
                r+=1



            mms = MinMaxScaler()
            mms.fit(df_[[column]])

            W_List[column].append(wasserstein_distance( np.squeeze(mms.transform(df_synth[[column]]))
                                               ,np.squeeze(mms.transform(df_[[column]]))
                                              )
                         )
    

In [ ]:
uni_plots3(title = "Adults FULL",
df_synth = synthetic_full,
df_ = X_test_t2)

In [ ]:
uni_plots2(title = "Adults FULL",
df_synth = synthetic_full,
df_ = X_test_t2, c_size = 3, r_size = 2)

## 07.C - Utilidade

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import roc_auc_score


from sklearn.preprocessing import OrdinalEncoder
new_df2 = X_test_t2.copy()

enc = OrdinalEncoder()
new_df2[X_test_t2.select_dtypes(include = ['object']).columns] = enc.fit_transform(X_test_t2.select_dtypes(include = ['object']).copy())

base_olga = pd.DataFrame()

base = ['Adults Full', 'Adults Outliers 1.5', 'Adults Outliers 3.0']
i = 0

for data_temp in [synthetic_full, synthetic_out15, synthetic_out30]:
    RX_m = new_df2.drop("salary", axis = 1).copy()
    RY_m = new_df2[['salary']].copy()

    X_trainR, X_testR, Y_trainR, Y_testR  = train_test_split(RX_m, RY_m , test_size=0.2, random_state=42)

    RFC_R = RandomForestClassifier()
    RFC_R.fit(X_trainR,Y_trainR)

    
    synth = data_temp.dropna().copy()
    synth[synth.select_dtypes(include = ['object']).columns] = enc.transform(synth.select_dtypes(include = ['object']).copy())

    
    SX_m = synth.drop("salary", axis = 1).copy()
    SY_m = synth[['salary']].copy()


    # X_trainS = SX_m.loc[X_trainR.index,:]
    # X_testS  = SX_m.loc[X_testR.index,:]
    # Y_trainS = SY_m.loc[Y_trainR.index,:]
    # Y_testS  = SY_m.loc[Y_testR.index,:]

    X_trainS, X_testS, Y_trainS, Y_testS  = train_test_split(SX_m, SY_m , test_size=0.2, random_state=42)

    RFC_S = RandomForestClassifier()
    RFC_S.fit(X_trainS,Y_trainS)


    #Treino Real, Teste Real
    final_predict_RR = pd.DataFrame({'Y':Y_testR.salary.values
                                     ,"RR_pred":RFC_R.predict(X_testR)
                                     ,"RR_prob":RFC_R.predict_proba(X_testR)[:,1]}, index = X_testR.index)


    #Treino Sintético, Teste Sintético
    final_predict_SS = pd.DataFrame({'Y':Y_testS.salary.values,
                                     "SS_pred":RFC_S.predict(X_testS),
                                     "SS_prob":RFC_S.predict_proba(X_testS)[:,1]}, index = X_testS.index)


    #Treino Sintético, Teste Real
    final_predict_SR = pd.DataFrame({'Y':Y_testR.salary.values,
                                     "SR_pred":RFC_S.predict(X_testR),
                                     "SR_prob":RFC_S.predict_proba(X_testR)[:,1]}, index = X_testR.index)

    final_predict_SR['SR_pred'] = final_predict_SR['SR_pred'].astype(int)
    final_predict_SS[['Y','SS_pred']] = final_predict_SS[['Y','SS_pred']].astype(int)

    
    testR = pd.concat( [X_testR, Y_testR, final_predict_RR, final_predict_SR[['SR_pred','SR_prob']]], axis = 1).copy()
    testR['model'] = 'test'
    testS = pd.concat( [X_testS, Y_testS,final_predict_SS], axis = 1).copy()
    testS['model'] = 'test'
    trainR = pd.concat( [X_trainR, Y_trainR], axis = 1).copy()
    trainR['model'] = 'train'
    trainS = pd.concat( [X_trainS, Y_trainS], axis = 1).copy()
    trainS['model'] = 'train'

    R = pd.concat([trainR,testR], axis = 0)
    R['type'] = 'Real'
    R['base'] = base[i]
    R['sint_model'] = 'VAE2'


    S = pd.concat([trainS,testS], axis = 0)
    S['type'] = 'Synt'
    S['base'] = base[i]
    S['sint_model'] = 'VAE2'
    i+=1
    
    base_olga = base_olga.append(R)
    base_olga = base_olga.append(S)
    
    
    df_m = pd.DataFrame([])
    metricas = [accuracy_score, f1_score, precision_score, recall_score, roc_auc_score]
    for m in metricas:
        if m.__name__ != "roc_auc_score":
            temp = pd.DataFrame(
                    {
                        "RR":m(final_predict_RR.Y, final_predict_RR.RR_pred),
                        "SS":m(final_predict_SS.Y, final_predict_SS.SS_pred),
                        "SR":m(final_predict_SR.Y, final_predict_SR.SR_pred)
                    }, index = [m.__name__])
            df_m = df_m.append(temp)
        else:
            temp = pd.DataFrame(
                    {
                        "RR":m(final_predict_RR.Y, final_predict_RR.RR_prob),
                        "SS":m(final_predict_SS.Y, final_predict_SS.SS_prob),
                        "SR":m(final_predict_SR.Y, final_predict_SR.SR_prob)
                    }, index = [m.__name__])
            df_m = df_m.append(temp)

    print(df_m)

# VAE 5HL

## 07.D - Privacidade

In [ ]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OrdinalEncoder
new_df2 = X_test_t2.copy()

enc = OrdinalEncoder()
new_df2[X_test_t2.select_dtypes(include = ['object']).columns] = enc.fit_transform(X_test_t2.select_dtypes(include = ['object']).copy())

new_df3 = new_df2.copy()
enc2 = MinMaxScaler()
new_df3 = pd.DataFrame(enc2.fit_transform(new_df2.copy()), columns = new_df2.columns)

base = ['Adults Full', 'Adults Outliers 1.5', 'Adults Outliers 3.0']
i = 0

base_dist = pd.DataFrame()

for data_temp in [synthetic_full, synthetic_out15, synthetic_out30]:
    synth = data_temp.dropna().copy()
    synth[synth.select_dtypes(include = ['object']).columns] = enc.transform(synth.select_dtypes(include = ['object']).copy())
    synth3 = pd.DataFrame(enc2.transform(synth.copy()), columns = synth.columns, index = synth.index)
    
    from sklearn.neighbors import NearestNeighbors

    neigh = NearestNeighbors(n_neighbors=2, radius=0.6)
    neigh.fit(new_df3)
    distances=map(lambda x: neigh.kneighbors(synth3.iloc[[x]], 2, return_distance=True)[0][0], range(synth3.shape[0]))
    ids=map(lambda x: neigh.kneighbors(synth3.iloc[[x]], 2, return_distance=True)[1][0], range(synth3.shape[0]))
    distances_l = pd.DataFrame(list(distances), index = synth3.index)
    ids_l = pd.DataFrame(list(ids), index = synth3.index)
    
    temp = pd.concat([distances_l,ids_l], axis = 1)
    temp['base'] = base[i]
    i+=1
    
    base_dist = base_dist.append(temp)
    
    print(pd.DataFrame(distances_l[0].append(distances_l[1]).quantile(q = [0,0.05,0.25,0.5,0.75,1])).T)

In [ ]:
base_dist.columns = ['distancia_1','distancia_2','id_proximo_1','id_proximo_2','base']
base_dist.reset_index().to_csv('base_distancia_AE.csv', index = False)
base_olga.reset_index().to_csv('base_score_modelo_AE.csv', index = False)